In [1]:
import os

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print(cwd)


/Users/co2ac/Documents/dsp_23may/toGit3


In [2]:
from PIL import Image
import os

# Set the path of the folder containing the subfolders
folder_path = "/Users/co2ac/Documents/dsp_23may/mts_2_twentyEight"

# Set the path of the output folder
output_path = "/Users/co2ac/Documents/dsp_23may"

# Loop through all the subfolders in the folder
for subfolder_name in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder_name)
    
    # Check if the subfolder is a directory
    if os.path.isdir(subfolder_path):
        # Create a new subfolder in the output folder with the same name as the subfolder
        output_subfolder_path = os.path.join(output_path, subfolder_name)
        os.makedirs(output_subfolder_path, exist_ok=True)
        
        # Loop through all the images in the subfolder
        for filename in os.listdir(subfolder_path):
            # Check if the file is an image
            if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
                # Open the image
                img = Image.open(os.path.join(subfolder_path, filename))

                # Resize the image to 28x28
                img = img.resize((28, 28))

                # Save the resized image to the output subfolder with the same filename
                output_filename = os.path.join(output_subfolder_path, filename)
                img.save(output_filename)


In [4]:
import os
from PIL import Image

# Set the path of the folder containing the images
folder_path = "/Users/co2ac/Documents/dsp_23may/mts_2_tE_2"

# Loop through all the subfolders in the folder
for subfolder_name in os.listdir(folder_path):
    # Get the full path of the subfolder
    subfolder_path = os.path.join(folder_path, subfolder_name)
    # Check if the subfolder is a directory
    if os.path.isdir(subfolder_path):
        # Loop through all the images in the subfolder
        for filename in os.listdir(subfolder_path):
            # Check if the file is an image
            if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
                # Open the image and convert it to grayscale
                img = Image.open(os.path.join(subfolder_path, filename)).convert("L")

                # Verify that the image has been resized to 28x28 pixels
                width, height = img.size
                if width != 28 or height != 28:
                    print(f"{os.path.join(subfolder_path, filename)} is not 28x28 pixels")

print("Verification complete.")


Verification complete.


## Convert into CSV

In [5]:
import pandas as pd
import os

# Set the path of the folder containing the 10 subfolders
folder_path = "/Users/co2ac/Documents/dsp_23may/mts_2_tE_2"

# Initialize an empty list to store the image data
image_data = []

# Loop through all the subfolders in the folder
for subfolder_name in os.listdir(folder_path):
    # Get the full path of the subfolder
    subfolder_path = os.path.join(folder_path, subfolder_name)
    # Check if the subfolder is a directory
    if os.path.isdir(subfolder_path):
        # Loop through all the images in the subfolder
        for filename in os.listdir(subfolder_path):
            # Check if the file is an image
            if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
                # Open the image and convert it to grayscale
                img = Image.open(os.path.join(subfolder_path, filename)).convert("L")
        
                # Flatten the image into a 1D array
                pixel_values = list(img.getdata())
                
                # Add the flattened image and its class to the image data list
                image_data.append(pixel_values + [subfolder_name])

# Convert the image data list into a pandas DataFrame and save it to a CSV file
df = pd.DataFrame(image_data)
df.to_csv("output3.csv", index=False, header=False)



## Deployment of model on preprocessed data

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CSV file into a pandas DataFrame
df = pd.read_csv("output.csv", header=None)

# Split the DataFrame into input (pixel values) and output (class labels)
X = df.iloc[:, :-1].values.astype(np.float32)
y = df.iloc[:, -1].values.astype(np.float32)

# Reshape the input data to 28x28 pixels
X = X.reshape(-1, 28, 28, 1)

# Define an ImageDataGenerator object to augment and preprocess the data
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=1./255,
)

# Use the ImageDataGenerator to load the data in batches
batch_size = 32
train_iterator = datagen.flow(
    X, y, batch_size=batch_size, shuffle=True
)

# Train your CNN model using the train_iterator
model.fit(train_iterator, epochs=10, steps_per_epoch=len(X) // batch_size)
